In [1]:
from Bio import SeqIO
import fnmatch
from Bio.Seq import Seq
from Bio import SearchIO
from Bio import motifs
from Bio.SeqRecord import SeqRecord
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import re
from pathlib import Path
import gzip
from itertools import islice
import shutil
from Bio.Blast import NCBIXML
import os
import csv
import time
from datetime import datetime
import statistics

In [2]:
# importing reads
def import_fastq_files_current_directory(end_file):
    sequence_vars = {}  # Dictionary to hold sequences with variable names as keys
    sequence_var_names = []  # List to hold the variable names

    # Get the current directory
    current_directory = os.getcwd()

    # Iterate over all files in the current directory
    for filename in os.listdir(current_directory):
        if filename.endswith(end_file):
            # Extract the variable name by removing the ".fastq.gz" extension
            var_name = filename.replace(end_file, "")
            
            # Create the full file path
            file_path = os.path.join(current_directory, filename)
            
            # Read the FASTQ file using SeqIO and store it in the variable
            with gzip.open(file_path, "rt") as handle:
                sequence_vars[var_name] = list(SeqIO.parse(handle, "fastq"))
            
            # Append the variable name to the list of sequence variable names
            sequence_var_names.append(var_name)
    
    return sequence_vars, sequence_var_names

In [3]:
os.chdir("/Users/glampe/Desktop/Columbia/Sternberg_Data/PACE_updates/20240903-indels/20240903-HTS-indelcalc/Edited-trimmed-alleles")
sequences, sequence_names = import_fastq_files_current_directory("_TnTrimmed_R1.fastq.gz")
os.chdir("/Users/glampe/Desktop/Columbia/Sternberg_Data/PACE_updates/20240903-indels/20240903-HTS-indelcalc/Edited-alleles")
sequences_untrimmed, sequence_names_untrimmed = import_fastq_files_current_directory("_edited_R1.fastq.gz")

In [20]:
# checking input reads
for item in sequence_names:
    print(item, len(sequences[item]),len(sequences_untrimmed[item]))

TRL25-HEK-WT-Rep1 10 10
TRL25-AAVS1-eeCAST-Rep2 6815 6815
TRL25-hROSA26-eeCAST-Rep1 9418 9418
TRL25-HEK-WT-Rep3 12 12
TRL25-hROSA26-eeCAST-Rep3 12186 12186
TRL25-AAVS1-WT-Rep2 358 358
TRL25-ALB-WT-Rep1 45 45
TRL25-hROSA26-WT-Rep1 150 150
TRL25-ALB-eeCAST-Rep2 10805 10805
TRL25-HEK-eeCAST-Rep2 1428 1428
TRL25-hROSA26-WT-Rep3 234 234
TRL25-ALB-WT-Rep3 92 92
TRL25-AAVS1-eeCAST-Rep3 9061 9061
TRL25-AAVS1-WT-Rep1 427 427
TRL25-hROSA26-eeCAST-Rep2 11462 11462
TRL25-AAVS1-eeCAST-Rep1 7954 7954
TRL25-AAVS1-WT-Rep3 321 321
TRL25-HEK-WT-Rep2 33 33
TRL25-HEK-eeCAST-Rep1 1617 1617
TRL25-ALB-eeCAST-Rep1 11716 11716
TRL25-hROSA26-WT-Rep2 161 161
TRL25-ALB-WT-Rep2 64 64
TRL25-ALB-eeCAST-Rep3 13727 13727
TRL25-HEK-eeCAST-Rep3 1844 1844


In [21]:
# generate qscore passing for minimum base call score
def passes_quality_filter(seqrecord, threshold):
    # Convert base qualities to Boolean based on Qscore threshold value. Only use reads with >=50% non-N:
    # recordqual = [x > threshold for x in seqrecord.letter_annotations['phred_quality']]  # list of True, False etc
    # return float(sum(recordqual)) / float(len(recordqual)) >= .5  # note that True = 1, False- = 0 for summing
    # modified to just look for min q score present
    return min(seqrecord.letter_annotations['phred_quality'])>=threshold

In [22]:
# qscore filtering
sequences_qscore = {}
untrimmed_qscore = {}
for item in sequence_names:
    sequences_qscore[item]=[]
    untrimmed_qscore[item]=[]
    read_ids=[]
    for sequence in sequences_untrimmed[item]:
        if passes_quality_filter(sequence,20):
            read_ids.append(sequence.id)
            untrimmed_qscore[item].append(sequence)
    for sequence in sequences[item]:
        if sequence.id not in read_ids: continue
        if passes_quality_filter(sequence,20):
            sequences_qscore[item].append(sequence)


In [23]:
# check the amount of reads that passed qScore
for item in sequence_names:
    print(item, len(sequences_qscore[item]),len(untrimmed_qscore[item]))

TRL25-HEK-WT-Rep1 2 2
TRL25-AAVS1-eeCAST-Rep2 1568 1568
TRL25-hROSA26-eeCAST-Rep1 1096 1096
TRL25-HEK-WT-Rep3 1 1
TRL25-hROSA26-eeCAST-Rep3 1489 1489
TRL25-AAVS1-WT-Rep2 83 83
TRL25-ALB-WT-Rep1 21 21
TRL25-hROSA26-WT-Rep1 17 17
TRL25-ALB-eeCAST-Rep2 6754 6754
TRL25-HEK-eeCAST-Rep2 418 418
TRL25-hROSA26-WT-Rep3 29 29
TRL25-ALB-WT-Rep3 56 56
TRL25-AAVS1-eeCAST-Rep3 2072 2072
TRL25-AAVS1-WT-Rep1 106 106
TRL25-hROSA26-eeCAST-Rep2 1515 1515
TRL25-AAVS1-eeCAST-Rep1 1855 1855
TRL25-AAVS1-WT-Rep3 56 56
TRL25-HEK-WT-Rep2 6 6
TRL25-HEK-eeCAST-Rep1 494 494
TRL25-ALB-eeCAST-Rep1 7373 7373
TRL25-hROSA26-WT-Rep2 27 27
TRL25-ALB-WT-Rep2 35 35
TRL25-ALB-eeCAST-Rep3 8547 8547
TRL25-HEK-eeCAST-Rep3 572 572


In [24]:
# generate reference sequences and the anchor sequences to start the comparison

target_sequences = {
    'AAVS1':'accgCCCCAAGGATCTCCCGGTCCCCGCCCGGCGTGCTGACGTCACGGCGCTGCCCCAGGGTGTGCTGGGCAGGTCGCGGGGAGCGCTGGGAAATGGAGTCCATTAGCAGAAGTGGCCCTTGGCCACTTCCAGGAGTCGCTGTGCCCCGATGCACACTGGGAAGTCCGCAGCTCCGAGGCGCCCAGTGGAAATCGCCAGATGAGGGCCTCCTCCGGGGAATGCTGGGAAATGGAGTCTACAGGCCGGAGGGGTGCCCCACGGCATACTAGGAAGTGTGTAGCACCGGGTAAAGGGGATG'.upper(),
    'ALB':'CCCAAAGACCTATCCATTGCACTATGCTTTATTTAAAAACCACAAAACCTGTGCTGTTGATCTCATAAATAGAACTTGTATTTATATTTATTTTCATTTTAGTCTGTCTTCTTGGTTGCTGTTGATAGACACTAAAAGAGTATTAGATATTATCTAAGTTTGAATATAAGGCtataaatatttaataatttttaaaataGTATTCTTGGTAATTGAATTATTCTTCTGTTTAAAGGCAGAAGAAATAATTGAACATCATCCTGAGTTTTTCTGT'.upper(),
    'HEK':'GCTTTTCCTAGACAGGGGCTAGTATGTGCAGCTCCTGCACCGGGATACTGGTTGACAAGTTTGGCTGGGCTGGAAGCCAGCACCTAGGGAGGTCCCTGGAAGGGGCCAGCCTCACCAGGAGAGGAGGGACCTGGCCCTTCAGGGTCGAGCTCAACAGAGGAAAAGATCTCAGGGCACCCAGAGCCCAGTGGCTTTCAGCACCTGCATGAAAATCAGAGATC'.upper(),
    'hROSA26':'CGTGGGAAGTCGGGAACATAATGTTTGTTACGTTGGGAGGGAAAGGGGTGGCTGGATGCAGGCGGGAGGGAGGCCCGCCCTGCGGCAACCGGAGGGGGAGGGAGAAGGGAGCGGAAAATGCTCGAAACCGGACGGAGCCATTGCTCTCGCAGAGGGAGGAGCGCTTCCGGCTAGCCTCTTGTCGCCGATTGGCCGTTTCTCCTCCCGCCGTGTGTGAAAACACAAATGGCGTATTCTGGTTGGAGTAAAGCTCCTGTCAGTTACGCCGTC'.upper()
}
crRNAs = {
    'AAVS1':'TGGGAAATGGAGTCCATTAG',
    'ALB':'TTATATTTATTTTCATTTTA',
    'HEK':'TTTCCTAGACAGGGGCTAGT',
    'hROSA26':'GGAGCGGAAAATGCTCGAAA'
}
anchors = {}
for item in crRNAs.keys():
    anchors[item]=crRNAs[item][-20:-5]
print(anchors)

{'AAVS1': 'TGGGAAATGGAGTCC', 'ALB': 'TTATATTTATTTTCA', 'HEK': 'TTTCCTAGACAGGGG', 'hROSA26': 'GGAGCGGAAAATGCT'}


In [25]:
# rev function 
def reverse_complement_sequences(sequence_list):
    reverse_complemented_list = []
    
    for record in sequence_list:
        # Reverse complement the sequence
        reverse_complemented_seq = record.seq.reverse_complement()
        
        # Create a new SeqRecord with the reverse complement sequence
        reverse_complemented_record = SeqRecord(reverse_complemented_seq, 
                                                id=f"{record.id}_revcomp")
        
        # Append the reverse complemented record to the list
        reverse_complemented_list.append(reverse_complemented_record)
    
    return reverse_complemented_list

In [26]:
# rev comp all the reads to deal with orientation of sequencing
sequences_rev = {}
untrimmed_rev = {}
for item in sequence_names:
    sequences_rev[item]=reverse_complement_sequences(sequences_qscore[item])
    untrimmed_rev[item]=reverse_complement_sequences(untrimmed_qscore[item])
#     print(sequences_rev[item][0])
#     print(untrimmed_rev[item][0])
#     break

In [27]:
# filter for reads with the anchor and trim there to make it easy
anchored_seq={}
anchored_untrimmed={}
for item in sequence_names:
    anchored_seq[item]=[]
    anchored_untrimmed[item]=[]
    site = ''
    for anchorsite in anchors.keys():
        if anchorsite in item:
            site = anchorsite
            break
#     print(item, site)
    for record in sequences_rev[item]:
        anchorbase=record.seq.find(anchors[site])
        if anchorbase != -1:
            trimmed_sequence = record.seq[anchorbase:]
            trimmed_record = SeqRecord(trimmed_sequence, id=record.id)
            anchored_seq[item].append(trimmed_record)

    for record in untrimmed_rev[item]:
        anchorbase=record.seq.find(anchors[site])
        if anchorbase != -1:
            trimmed_sequence = record.seq[anchorbase:]
            trimmed_record = SeqRecord(trimmed_sequence, id=record.id)
            anchored_untrimmed[item].append(trimmed_record)
            
    print(item, len(anchored_seq[item]), len(anchored_untrimmed[item]))

TRL25-HEK-WT-Rep1 0 0
TRL25-AAVS1-eeCAST-Rep2 1518 1518
TRL25-hROSA26-eeCAST-Rep1 1093 1093
TRL25-HEK-WT-Rep3 1 1
TRL25-hROSA26-eeCAST-Rep3 1488 1488
TRL25-AAVS1-WT-Rep2 64 64
TRL25-ALB-WT-Rep1 21 21
TRL25-hROSA26-WT-Rep1 17 17
TRL25-ALB-eeCAST-Rep2 6741 6741
TRL25-HEK-eeCAST-Rep2 416 416
TRL25-hROSA26-WT-Rep3 29 29
TRL25-ALB-WT-Rep3 55 55
TRL25-AAVS1-eeCAST-Rep3 1993 1993
TRL25-AAVS1-WT-Rep1 94 94
TRL25-hROSA26-eeCAST-Rep2 1510 1510
TRL25-AAVS1-eeCAST-Rep1 1814 1814
TRL25-AAVS1-WT-Rep3 48 48
TRL25-HEK-WT-Rep2 2 2
TRL25-HEK-eeCAST-Rep1 487 487
TRL25-ALB-eeCAST-Rep1 7356 7356
TRL25-hROSA26-WT-Rep2 27 27
TRL25-ALB-WT-Rep2 30 30
TRL25-ALB-eeCAST-Rep3 8536 8536
TRL25-HEK-eeCAST-Rep3 563 563


In [28]:
# need to trim the stock sequence too, then go base-by-base
targets_trimmed={}

for item in target_sequences.keys():
    sequence = Seq(target_sequences[item])
    anchorsite=sequence.find(anchors[item])
    targets_trimmed[item]=sequence[anchorsite:]

print(targets_trimmed)
#     print(item, site)
#     for record in sequences_rev[item]:
#         anchorbase=record.seq.find(anchors[site])
#         if anchorbase != -1:
#             sequences_trimmed[item].append(record.seq[anchorbase:])

#     print(item, len(sequences_trimmed[item]))

{'AAVS1': Seq('TGGGAAATGGAGTCCATTAGCAGAAGTGGCCCTTGGCCACTTCCAGGAGTCGCT...ATG'), 'ALB': Seq('TTATATTTATTTTCATTTTAGTCTGTCTTCTTGGTTGCTGTTGATAGACACTAA...TGT'), 'HEK': Seq('TTTCCTAGACAGGGGCTAGTATGTGCAGCTCCTGCACCGGGATACTGGTTGACA...ATC'), 'hROSA26': Seq('GGAGCGGAAAATGCTCGAAACCGGACGGAGCCATTGCTCTCGCAGAGGGAGGAG...GTC')}


In [78]:
# now time to go base by base, and count it as needed, need to fix the indexing to be right for the tn insertion
# just go 10 bp in, that will account for tRL or tLR

# UNIVERSAL tn sequence TGTCGCTGCA
# read by read, each read has 3 things: 
# the upstream sequence before trimming
# the sequence after trimming
# wt reference sequence

# dictionaries are: 
# anchored_seq={}
# anchored_untrimmed={}

# the upstream bases are indexed as 0-len(upstream)-15
# insertion are counted 1 to 10

base_list = ['A','T','C','G']

tn_end = Seq('TGTCGCTGCA')

read_counter = {}
error_counter = {}
mismatch_description = {}
mismatch_from_donor = {}
totals_for_mismatch = {}
totals_for_donor = {}
reverseflank = Seq('TGTTTT')# reverse, not rev comp, to just split right in the dict
# position 5 will be revflank[5-1]

#### Mismatch description design:
# mismatch_description[sample] is dict of positions
# each position is 3 incorrect bases followed by dictionary of observed values

for item in sequence_names:
    mismatch_description[item] = {}
    mismatch_from_donor[item] = {}
    totals_for_donor[item]={}
    for position in [1,2,3,4,5]:
        totals_for_donor[item][position]=0
        mismatch_from_donor[item][position] = {}
        for baseseq in base_list: mismatch_from_donor[item][position][baseseq]=0
        
    totals_for_mismatch[item] = {}
    print(item)
    for target_site in targets_trimmed:
        if target_site in item:
            target=target_site
            break
    counter = {}
    depth_count = {}
    if len(anchored_seq[item])<1: continue
    read_counter[item]={}
    error_counter[item]={}
    for record in anchored_seq[item]:
        upstream_distance=0-len(record.seq)
        indexing=0 
        index=upstream_distance
        for base in record:
            if indexing not in mismatch_description[item]:
                bases = [basecall for basecall in base_list if basecall != targets_trimmed[target][indexing]]
                mismatch_description[item][indexing]= {}
                totals_for_mismatch[item][indexing] = 0
                for baseseq in bases: mismatch_description[item][indexing][baseseq]=0
            if index not in counter:
                counter[index]=0
                depth_count[index]=0

            depth_count[index]+=1
            if base != targets_trimmed[target][indexing]:
                totals_for_mismatch[item][indexing] += 1

                counter[index]+=1
                mismatch_description[item][indexing][record[indexing]]+=1
                upstream = -1*index
                if upstream < 6:
                    totals_for_donor[item][upstream]+=1
                    mismatch_from_donor[item][upstream][base] +=1
                
            index+=1 
            indexing+=1
        for record1 in anchored_untrimmed[item]:
            if record1.id != record.id: continue
            index2=0
            for base in record1:
                if index2<indexing:
                    index2+=1
                    continue
                position_counter = index+1
                if position_counter not in counter:
                    counter[position_counter]=0
                    depth_count[position_counter]=0
                depth_count[position_counter]+=1
                if tn_end[index] != base:
                    counter[position_counter]+=1

                index2+=1
                index+=1
                if index2>(indexing+9): break

                
            
    counter = dict(sorted(counter.items()))
    depth_count = dict(sorted(depth_count.items()))
    error_counter[item]=counter
    read_counter[item]=depth_count
    
    
#     print(counter)
#     print(depth_count)
#     break
    



#     error_counter[item]=counter
#     read_counter[item]=depth_count
# #     print(counter)
# #     print(depth_count)
# #     break
# # dictionary, each base from 1 to n, then have each target site be it's own output sheet, have two counts, mismatches and total counts at that base

TRL25-HEK-WT-Rep1
TRL25-AAVS1-eeCAST-Rep2
TRL25-hROSA26-eeCAST-Rep1
TRL25-HEK-WT-Rep3
TRL25-hROSA26-eeCAST-Rep3
TRL25-AAVS1-WT-Rep2
TRL25-ALB-WT-Rep1
TRL25-hROSA26-WT-Rep1
TRL25-ALB-eeCAST-Rep2
TRL25-HEK-eeCAST-Rep2
TRL25-hROSA26-WT-Rep3
TRL25-ALB-WT-Rep3
TRL25-AAVS1-eeCAST-Rep3
TRL25-AAVS1-WT-Rep1
TRL25-hROSA26-eeCAST-Rep2
TRL25-AAVS1-eeCAST-Rep1
TRL25-AAVS1-WT-Rep3
TRL25-HEK-WT-Rep2
TRL25-HEK-eeCAST-Rep1
TRL25-ALB-eeCAST-Rep1
TRL25-hROSA26-WT-Rep2
TRL25-ALB-WT-Rep2
TRL25-ALB-eeCAST-Rep3
TRL25-HEK-eeCAST-Rep3


In [70]:
file_name='evoCAST_junction_combined.xlsx'

min_negative_value = min(min(error_counter[sample].keys()) for sample in error_counter.keys())  # Capture min negative key

max_index = 10  # Set the maximum positive index

with pd.ExcelWriter(file_name) as writer:
    sheet_name="pooled"
    dataframe_dict = {"Base": list(range(min_negative_value, max_index + 1))}
    for sample in error_counter.keys(): 
        sample_column = [error_counter[sample].get(i, 0) for i in range(min_negative_value, max_index + 1)]
        dataframe_dict[sample] = sample_column

    for sample in error_counter.keys():
        sample_column = [read_counter[sample].get(i, 0) for i in range(min_negative_value, max_index + 1)]
        dataframe_dict[f"{sample}_read-depth"] = sample_column

    df = pd.DataFrame(dataframe_dict)
    df.to_excel(writer, sheet_name, index=False)

In [83]:
# print(mismatch_description)
print(totals_for_donor)

{'TRL25-HEK-WT-Rep1': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-AAVS1-eeCAST-Rep2': {1: 25, 2: 13, 3: 14, 4: 4, 5: 1}, 'TRL25-hROSA26-eeCAST-Rep1': {1: 4, 2: 1, 3: 2, 4: 2, 5: 0}, 'TRL25-HEK-WT-Rep3': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-hROSA26-eeCAST-Rep3': {1: 2, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-AAVS1-WT-Rep2': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-ALB-WT-Rep1': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-hROSA26-WT-Rep1': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-ALB-eeCAST-Rep2': {1: 131, 2: 57, 3: 99, 4: 14, 5: 10}, 'TRL25-HEK-eeCAST-Rep2': {1: 13, 2: 1, 3: 0, 4: 0, 5: 0}, 'TRL25-hROSA26-WT-Rep3': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-ALB-WT-Rep3': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-AAVS1-eeCAST-Rep3': {1: 13, 2: 8, 3: 9, 4: 8, 5: 2}, 'TRL25-AAVS1-WT-Rep1': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-hROSA26-eeCAST-Rep2': {1: 2, 2: 0, 3: 1, 4: 1, 5: 1}, 'TRL25-AAVS1-eeCAST-Rep1': {1: 31, 2: 20, 3: 17, 4: 6, 5: 6}, 'TRL25-AAVS1-WT-Rep3': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}, 'TRL25-HEK-WT-Rep2':

In [76]:
import pandas as pd

def create_mismatch_table(mismatch_description, totals_for_mismatch, output_file):
    # Initialize the ExcelWriter object to write multiple sheets
    with pd.ExcelWriter(f"/Users/glampe/Desktop/Columbia/Sternberg_Data/PACE_updates/20240903-indels/20240903-HTS-indelcalc/{output_file}.xlsx", engine='xlsxwriter') as writer:
        # Iterate through each sample in mismatch_description
        for sample_name, sample_data in mismatch_description.items():
            # Process only samples that contain the word "eeCAST"
            if "eeCAST" in sample_name:
                # Initialize a list to hold rows of the table
                table_data = []
                
                # Iterate through positions 30 to 55
                for position in range(30, 56):
                    # Initialize a row with the position number and default base counts
                    row = {
                        'Position': position,
                        'A': sample_data.get(position, {}).get('A', 0),
                        'T': sample_data.get(position, {}).get('T', 0),
                        'C': sample_data.get(position, {}).get('C', 0),
                        'G': sample_data.get(position, {}).get('G', 0),
                        'Total': totals_for_mismatch.get(position, 0)  # Use totals_for_mismatch for total count
                    }
                    
                    # Append the row to the table data
                    table_data.append(row)
                
                # Convert the list of rows to a DataFrame
                df = pd.DataFrame(table_data)
                
                # Write the DataFrame to a new sheet in the Excel file
                df.to_excel(writer, sheet_name=sample_name, index=False)
        
    print(f"All tables saved to {output_file}.xlsx")

# Example usage
create_mismatch_table(mismatch_description, totals_for_mismatch, "output")


All tables saved to output.xlsx


In [84]:
import pandas as pd
import re

def create_mismatch_table_normalized_ignore_wt(mismatch_description, totals_for_mismatch, output_file):
    # Dictionary to store data for each sample (grouped by name before "Rep#")
    grouped_data = {}

    # Iterate through each sample in mismatch_description
    for sample_name, sample_data in mismatch_description.items():
        # Skip samples that contain "WT"
        if "WT" in sample_name:
            continue

        # Extract the sample name without "Rep#" part using regex
        base_sample_name = re.sub(r'-Rep\d+', '', sample_name)

        # Initialize or append to the grouped data for each sample
        if base_sample_name not in grouped_data:
            grouped_data[base_sample_name] = []

        # Collect data for each position (1 to 5 for this example)
        for position in range(1, 6):
            # Get total for this position, default to 1 if missing or zero to avoid division by zero
            total = totals_for_mismatch[sample_name].get(position, 1)

            # Initialize a row with normalized base counts
            row = {
                'Sample': sample_name,
                'Position': position,
                'A': sample_data.get(position, {}).get('A', 0) / total if total > 0 else 0,
                'T': sample_data.get(position, {}).get('T', 0) / total if total > 0 else 0,
                'C': sample_data.get(position, {}).get('C', 0) / total if total > 0 else 0,
                'G': sample_data.get(position, {}).get('G', 0) / total if total > 0 else 0,
                'Total': total
            }

            # Append the row to the appropriate sample in grouped_data
            grouped_data[base_sample_name].append(row)

    # Initialize the ExcelWriter object to write multiple sheets
    with pd.ExcelWriter(f"{output_file}.xlsx", engine='xlsxwriter') as writer:
        # Write each sample (grouped by base name) into a separate sheet
        for base_sample_name, table_data in grouped_data.items():
            # Convert the list of rows to a DataFrame
            df = pd.DataFrame(table_data)

            # Write the DataFrame to a new sheet in the Excel file
            df.to_excel(writer, sheet_name=base_sample_name, index=False)

    print(f"All tables saved to {output_file}.xlsx")

# Example usage
create_mismatch_table_normalized_ignore_wt(mismatch_from_donor, totals_for_donor, "/Users/glampe/Desktop/Columbia/Sternberg_Data/PACE_updates/20240903-indels/20240903-HTS-indelcalc/output_grouped_normalized_ignore_wt")


All tables saved to /Users/glampe/Desktop/Columbia/Sternberg_Data/PACE_updates/20240903-indels/20240903-HTS-indelcalc/output_grouped_normalized_ignore_wt.xlsx
